In [5]:
from autoscalingsim import simulator
import pandas as pd

starting_time = pd.Timestamp("2020-09-17T10:00:00")
simulation_step = pd.Timedelta(100, unit = 'ms')
time_to_simulate = pd.Timedelta(10, unit = 'm')
config_dir = "experiments/short-experiment/holtwinters"
#"experiments/short-experiment/reactive-mapping"#"experiments/testazuremanual2"#"experiments/test"#
results_dir = None

simulator = simulator.Simulator(simulation_step, starting_time, time_to_simulate, 666)

simulator.add_simulation(config_dir, results_dir)

simulator.start_simulation()

6001it [16:16,  6.15it/s]                                                                                              


In [6]:
from stethoscope.analytical_engine import AnalysisFramework

af = AnalysisFramework(simulation_step, 'D:/AutoscalingSim/results/test/short_oscillating/holtwinters')
af.build_figures_for_single_simulation(simulator.simulations['holtwinters'], '')#af.build_figures_for_single_simulation(simulator.simulations['test'], '')#

<Figure size 432x288 with 0 Axes>

<Figure size 432x288 with 0 Axes>

In [ ]:
from training_ground.trainer import Trainer

t = Trainer("training_conf/mapping-sgd/load")
t.start_training()

In [ ]:
import tensorflow as tf
import numpy as np

xxx = tf.keras.models.load_model('D:/@TUM/PhD/FINAL/experimentresults/models/nn/service-75c4ce5c-4da4-11eb-b901-d8cb8af1e959/eu/group1/dav_model.mdl')
in_data = [[3., 1., 0.18705613], [3., 0., 0.17934185]]
xxx.predict(tf.constant(in_data, dtype=tf.float16))

In [ ]:
import numpy as np
import collections
from sklearn.linear_model import SGDRegressor
import pickle


model_config = {   "loss": "squared_loss",
                   "penalty": "l2",
                   "alpha": 0.0001,
                   "l1_ratio": 0.15,
                   "tol": 0.001,
                   "validation_fraction": 0.1,
                   "n_iter_no_change": 5,
                   "epsilon": 0.1 }

sgd_model = SGDRegressor(**model_config)

def formatter_function(cur_aspect_val, cur_metrics_vals):

    joint_vals = [[ val for val in cur_aspect_val ] if isinstance(cur_aspect_val, collections.Iterable) else [cur_aspect_val]]
    for metric_vals in cur_metrics_vals.values():
        joint_vals.append([ val for val in metric_vals ] if isinstance(metric_vals, collections.Iterable) else [metric_vals] )

    return np.asarray(joint_vals).T

cur_aspect_val = [1,2]
cur_metrics_vals = {'load': [10, 20], 'wt': [133, 122]}
X_train = formatter_function(cur_aspect_val, cur_metrics_vals)
y_train = np.asarray([1200, 1100])
print(X_train)
sgd_model.partial_fit(X_train, y_train)

X_test = formatter_function(1, {'load': 11, 'wt': 100})
print(X_test)
sgd_model.predict(X_test)

pickle.dump( sgd_model, open( "save.p", "wb" ) )

x = pickle.load( open( "save.p", "rb" ) )
x.get_params()

In [ ]:
from experimentgenerator.experiment_generator import ExperimentGenerator
a = ExperimentGenerator('experiments/testazure3')
a.generate_experiment('experiment_recipes/azure.json')

In [ ]:
from autoscalingsim.simevaluator.evaluation_framework import SimulationQualityEvaluationFramework

sqef = SimulationQualityEvaluationFramework('experiments/testevaluation/evaluator_conf.json')
sqef.create_simulations()
sqef.simulate()
sqef.build_figures('D:/AutoscalingSim/results/test')

In [ ]:
# To profile:
# python -m cProfile -o D:\AutoscalingSim\results\profiling_res.txt autoscalingsim-cl.py --step 10 --start "2020-09-17T10:00:00" --confdir "experiments/test" --simtime 1m
import pstats
from pstats import SortKey
p = pstats.Stats('D://AutoscalingSim//results//profiling_res.txt')
p.strip_dirs().sort_stats(SortKey.CALLS).print_stats()

In [ ]:
from cruncher.cruncher import Cruncher

c = Cruncher('cruncher_conf/forecasting_experiment/oscillating_load/')
c.run_experiment()

In [ ]:
import pandas as pd
import random

idx = pd.date_range(pd.Timestamp(1000, unit = 'ms'), pd.Timestamp(20000, unit = 'ms'), freq = pd.Timedelta(1000, unit = 'ms'))

x = pd.DataFrame({'datetime': idx, 'value': random.choices(range(10000), k = len(idx))}).set_index('datetime')
x.shift(-15).dropna()

In [ ]:
# Increasing pattern in load
import numpy as np
import json

interval_percentage = 6 / (5 * 60) # 6 seconds
step_rps = 3 / (5 * 60 / 6)
vals = []
for request_rate in np.arange(0, 3.00, step_rps):
    vals.append({ "requests_count_level": round(request_rate, 2), "percentage_of_interval": interval_percentage })
    
json.dumps(vals)

In [ ]:
from autoscalingsim import simulator
import pandas as pd
import collections
import time
import pickle

simulation_steps_ms = [10,20,30,40,50,60,70,80,90,100]
starting_time = pd.Timestamp("2020-09-17T10:00:00")
time_to_simulate = pd.Timedelta(10, unit = 'm')
repeats = 10

results = collections.defaultdict(list)
for sim_step_raw in simulation_steps_ms:
    print(f'current simulation step is {sim_step_raw} ms')
    for _ in range(repeats):
        start = time.time()
        simulation_step = pd.Timedelta(sim_step_raw, unit = 'ms')
        config_dir = "experiments/testazuremanual2"
        results_dir = None

        sim = simulator.Simulator(simulation_step, starting_time, time_to_simulate)

        sim.add_simulation(config_dir, results_dir)

        sim.start_simulation()
        
        results[sim_step_raw].append(time.time() - start)
        
pickle.dump( results, open( "performance_test_results_raw.pickle", "wb" ) )

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

res = pickle.load( open( "performance_test_results_raw.pickle", "rb" ) )

results_means = [np.mean(times_per_simstep) / (10 * 60) for times_per_simstep in res.values()]
results_stds = [np.std(times_per_simstep) / (10 * 60) for times_per_simstep in res.values()]
steps = np.arange(10, 101, 10)

p1 = plt.bar(steps, results_means, 7, yerr=results_stds)

plt.ylabel('Wall clock time per 1 simulated second, s')
plt.xlabel('Simulation step, ms')
plt.xticks(steps)
plt.yticks(np.arange(0.0, 2.0, 0.1))
plt.hlines(0.5, xmin = 5, xmax = 105, colors = 'r', linestyles = 'dashed')
#plt.show()

plt.savefig("./performance_results.png", dpi = 600, bbox_inches='tight')

# Diminishing returns:
(10 * 60_000) / steps